<a href="https://colab.research.google.com/github/iomicoml/Foods_recognition_/blob/main/model_food_recognition_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Fill model name. Fill information about conditions in which model was trained (count of images in dataset, classes count, time of training, etc.)**

In [1]:
ABOUT_MODEL_FILE_NAME = "INFO.txt"

info_about_model = """
  model_name: Faster-RCNN Inception-V2
  count_of_classes: 5
  average count_of_images per class: 150
  
  training_time: 2 hours
  final_loss_function_value: 0.05 - 0.09

  dataset: custom
  count_of_images: 750
  annotations_type: manual box bounding ( 'labelImg' free python application )
"""



# **To run all the cells sequentially just press "ctrl+F9"**

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import sys
import os
import glob
from google.colab.patches import cv2_imshow

from googleapiclient.http import MediaFileUpload

from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from datetime import date
today = date.today()

import re

In [3]:

from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/My Drive/Food_Recognition/Test/label_map_util.py" .
!cp "/content/drive/My Drive/Food_Recognition/Test/visualization_utils.py" .

!cp -r "/content/drive/My Drive/Food_Recognition/Test/inference_graph" .
!cp -r "/content/drive/My Drive/Food_Recognition/Test/object_detection" .
!cp -r "/content/drive/My Drive/Food_Recognition/Test/training" .
!cp -r "/content/drive/My Drive/Food_Recognition/Fridge_photos" .


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import label_map_util
import visualization_utils as vis_util


In [ ]:
# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Number of classes the object detector can identify
NUM_CLASSES = 5

# Dimension of images that will be saved in inference_test dir and showed below
DIM = (800, 600)


# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

images = glob.glob('Fridge_photos/' + '/*.jpg')

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.compat.v1.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')



# create new folder with date of inference testing ( (Iomicoml_acoount_drive) /Food_Recognition/Test_inference/)
folder_id = '18VzeTeOq-uCA3-V56acSYqbAf6VV4d_m'
file_metadata = {
    'name': today.strftime("%b-%d-%Y"),
    'mimeType': 'application/vnd.google-apps.folder',
    'parents': [folder_id]
}

file = drive_service.files().create(body=file_metadata,
                                    fields='id').execute()

test_inf_save_dir = file.get('id')


for img in images:
  image = cv2.imread(img)
  image = cv2.resize(image,DIM, interpolation=cv2.INTER_AREA) 
  image_rgb = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
  image_expanded = np.expand_dims(image_rgb, axis=0)

  # Perform the actual detection by running the model with the image as input
  (boxes, scores, classes, num) = sess.run(
      [detection_boxes, detection_scores, detection_classes, num_detections],
      feed_dict={image_tensor: image_expanded})

  # Draw the results of the detection (aka 'visulaize the results')

  vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      np.squeeze(boxes),
      np.squeeze(classes).astype(np.int32),
      np.squeeze(scores),
      category_index,
      use_normalized_coordinates=True,
      line_thickness=8,
      min_score_thresh=0.60)

  # All the results have been drawn on image. Now display the image.
  cv2_imshow(image)

  # current image name = Fridge_photos/()..).jpg, name for image write must be only ()..).jpg, part of string after /
  match = re.search('/(.+?)$', img)
  if match:
    img_name = match.group(1)
  else:
    img_name = img

  cv2.imwrite(img_name, image)

  file_metadata = {
  'name': img_name,
  'mimeType': 'image/jpeg',
  'parents': [test_inf_save_dir]
  }
  media = MediaFileUpload(img_name, 
                        mimetype='image/jpeg',
                        resumable=True)
  
  drive_service.files().create(body=file_metadata, media_body=media,fields='id').execute()

  # # Press any key to close the image
  # cv2.waitKey(0)
  # # Clean up
  # cv2.destroyAllWindows()

# create text_file for explanation model training conditions (top of notebook)


with open(ABOUT_MODEL_FILE_NAME, 'w') as out_file:
  out_file.write(info_about_model)

file_metadata = {
  'name': ABOUT_MODEL_FILE_NAME,
  'mimeType': 'text/plan',
  'parents': [test_inf_save_dir]
  }
media = MediaFileUpload(ABOUT_MODEL_FILE_NAME, 
                        mimetype='text/plan',
                        resumable=True)
  
drive_service.files().create(body=file_metadata, media_body=media,fields='id').execute()

os.remove(ABOUT_MODEL_FILE_NAME)


# remove all the .jpg files, that was created for save their copies in google drive inference_test dir
cur_dir_path = os.getcwd()
for filename in os.listdir(cur_dir_path):
    if filename.endswith(".jpg"):
        os.remove(filename)